# Tutorial 6: Segmentation-free analysis

In [ ]:
import numpy as np
import pandas as pd
import cv2
import argparse
import logging
from omics.constants import *
import os
import pickle
import re
import torch
import torch.utils.data as data
from copy import deepcopy
import random
from sklearn.preprocessing import LabelEncoder
from scipy.spatial import cKDTree
import scanpy as sc
from pytorch_lightning.callbacks import (EarlyStopping, LearningRateMonitor,
                                         ModelCheckpoint, Callback)
# import squidpy as sq
import torch.nn as nn
from math import exp
from sklearn.model_selection import train_test_split
import random
from argparse import Namespace
import h5py
from tqdm import tqdm
from omics.datasets.data_module import testDataModule as DataModule
import hashlib
import anndata as ad
from omics.Finetune.finetune_fold_83 import Omics
from pytorch_lightning import LightningModule, Trainer, seed_everything
from pytorch_lightning.callbacks import (EarlyStopping, LearningRateMonitor,
                                         ModelCheckpoint)
from argparse import ArgumentParser
import numpy as np
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import cv2
import logging
import joblib
import umap
from sklearn.manifold import TSNE
import json

from visualize import *

## Initialize configs and fix seed

In [ ]:
import os

# 设置环境变量
os.environ["OPENBLAS_NUM_THREADS"] = "8"
os.environ["MKL_NUM_THREADS"] = "8"
os.environ["OMP_NUM_THREADS"] = "8"
os.environ["NUMBA_NUM_THREADS"] = "8"
os.environ["OMP_DYNAMIC"] = "FALSE"
os.environ["MALLOC_ARENA_MAX"] = "2"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# 直接按照配置写死参数，避免 Notebook 再次解析 CLI
args = Namespace(
    ckpt_path="/media/bang/backup_omics/pretrained_model/continual_fold2_xenium_hbc1_rep2_2025_05_18_13_43_41/epoch=0-step=122700.ckpt",
    dataset_name="xenium_hbc1",
    config="/media/dang/Omics/omics/configs/bert_config_5-12.json",
    f=None,
    split_slice=None,
    target_width=7524,
    target_height=5469,
    x_min=0,
    x_max=7524,
    y_min=0,
    y_max=5469,
    sample_ratio=0.1,
    merge_threshold=0.5,
    percentile=70,
    pca_model_path="/media/dang/Omics/omics/visualize/xenium/results_for_sijie2/xenium_hbc1_None_-1_7522_4_5473/xenium_hbc1_None_-1_7522_4_5473_pca.pkl",
    color_model_path="/media/dang/Omics/omics/visualize/xenium/results_for_sijie2/xenium_hbc1_None_-1_7522_4_5473/xenium_hbc1_None_-1_7522_4_5473_pca_color.pkl",
    batch_id=0,  # 根据需要修改批次ID (0-9)
    total_batches=10
)

print(f"Processing batch {args.batch_id + 1}/{args.total_batches}")
print(f"ckpt_path: {args.ckpt_path}")
print(f"Dataset: {args.dataset_name}")

## Generate visualizations

In [ ]:

args.radius = 20
args.linear_hidden_dim = '256'
args.max_points = 20

# 加载数据集
split_slice = None
if args.dataset_name == 'mop1':
    adata = pd.read_pickle(MOUSE_SPOT_DATA1) 
    adata.rename(columns={'target_molecule_name': 'gene', 'cell_index': 'nucleus', 'slice_id': 'batch', 'global_x': 'x', 'global_y': 'y'}, inplace=True)
    adata['gene'] = adata['gene'].astype('category')
elif args.dataset_name == 'merfish':
    adata = pd.read_pickle(MERFISH_DATA1)
    split_slice = MERFISH_FOLDS[args.fold]['train']
elif args.dataset_name == 'seqfish':
    adata = pd.read_pickle(SEQFISH_DATA1)
elif args.dataset_name == 'AD_64g_m9721':
    # import ipdb; ipdb.set_trace()
    adata = pd.read_pickle(AD_64g_9721_DATA)
    adata.rename(columns={'slice': 'batch'}, inplace=True)
    # if args.fold:
    #     split_slice = AD_FOLDS[args.fold]['val']
elif args.dataset_name == 'AD_64g_m9781':
    adata = pd.read_pickle(AD_64g_9781_DATA)
    adata.rename(columns={'slice': 'batch'}, inplace=True)
elif args.dataset_name == 'AD_64g_m9919':
    adata = pd.read_pickle(AD_64g_9919_DATA)
elif args.dataset_name == 'AD_64g_m9930':
    adata = pd.read_pickle(AD_64g_9930_DATA)
elif args.dataset_name == 'AD_2766g_m9707':
    adata = pd.read_pickle(AD_2766g_9707_DATA)
elif args.dataset_name == 'AD_2766g_m9735':
    adata = pd.read_pickle(AD_2766g_9735_DATA)
elif args.dataset_name == 'AD_2766g_m9723':
    adata = pd.read_pickle(AD_2766g_9723_DATA)
elif args.dataset_name == 'AD_2766g_m9494':
    adata = pd.read_pickle(AD_2766g_9494_DATA)
elif args.dataset_name == 'AD_2766g_m11346':
    adata = pd.read_pickle(AD_2766g_11346_DATA)
elif args.dataset_name == 'AD_2766g_m11351':
    adata = pd.read_pickle(AD_2766g_11351_DATA)
elif args.dataset_name == 'AD_2766g_m9723_2':
    adata = pd.read_pickle(AD_2766g_9723_2_DATA)
elif args.dataset_name == 'xenium_hbc1':
    adata = pd.read_pickle(Xenium_hbc_rep1_DATA)
    split_slice = XENIUM_HBC_FOLDS1[args.fold]['train']
    split_slice += XENIUM_HBC_FOLDS1[args.fold]['val']
elif args.dataset_name == 'xenium_hbc1_rep2':
    adata = pd.read_pickle(Xenium_hbc_rep2_DATA)
    split_slice = XENIUM_HBC_FOLDS1[args.fold]['train']
    split_slice += XENIUM_HBC_FOLDS1[args.fold]['val']
else:
    raise NotImplementedError

# 加载模型
print(args.config)
model = Omics.load_from_checkpoint(**args.__dict__,checkpoint_path=args.ckpt_path,strict=False)
model.eval()
model = model.to('cuda')

# 数据预处理
if args.split_slice:
    if args.split_slice.isdigit():
        split_slice = [int(args.split_slice)]
    else:
        split_slice = [args.split_slice]

if split_slice:
    adata = adata.query("batch.isin(@split_slice)")

# 步骤1: 确定分辨率并处理spots
total_minx, total_maxx, total_miny, total_maxy = get_min_max_coordinates(adata)
scale_x, scale_y, pixel_size_x, pixel_size_y = determine_visualization_resolution(
    total_minx, total_maxx, total_miny, total_maxy, args.target_width, args.target_height)

# 坐标转换
x_min_prop = args.x_min / args.target_width
x_max_prop = args.x_max / args.target_width
y_min_prop = args.y_min / args.target_height
y_max_prop = args.y_max / args.target_height

x_range = total_maxx - total_minx
y_range = total_maxy - total_miny

args.x_min = int(total_minx + (x_min_prop * x_range))
args.x_max = int(total_minx + (x_max_prop * x_range))
args.y_min = int(total_miny + (y_min_prop * y_range))
args.y_max = int(total_miny + (y_max_prop * y_range))

# 过滤数据
if args.x_min is not None and args.x_max is not None and args.y_min is not None and args.y_max is not None:
    adata = adata.query("(x >= @args.x_min) & (x <= @args.x_max) & (y >= @args.y_min) & (y <= @args.y_max)")

# 分批处理spots
patch_adata = select_batch_spots(adata, args.batch_id, args.total_batches)

# 设置label encoder
label_encoder = LabelEncoder()
label_encoder.fit(adata['gene'].values)
annotation_num = patch_adata['gene'].unique().shape[0]

# 创建数据加载器
val_split = []
epoch_callback = EpochCallback()
dm = DataModule(SpatialRadiusDataset, my_collate_fn,
                data_pct=1.0, batch_size=1000, 
                num_workers=8, mask_ratio=0, radius=20,
                mask_function='random', dataset_name=args.dataset_name,
                max_points=20, train_split=split_slice, val_split=val_split,
                label_type='pretrain',callback=epoch_callback,
                x_min=args.x_min, x_max=args.x_max, y_min=args.y_min, y_max=args.y_max,
                adata=patch_adata, batch_id=args.batch_id)
dl = dm.inference_train_dataloader()

# 提取特征
train_rep = get_rep(dl, model)

# 创建保存前缀和统计文件路径
save_prefix = f"ficture_vis_results_final/{args.dataset_name}_{args.split_slice}_{args.x_min}_{args.x_max}_{args.y_min}_{args.y_max}"
os.makedirs("ficture_vis_results_final", exist_ok=True)

# 统一的统计信息文件路径
if args.dataset_name == 'xenium_hbc1_rep2':
    stats_file_path = f"global_stats_xenium_hbc1_p{args.percentile}.json"
else:
    stats_file_path = f"global_stats_{args.dataset_name}_p{args.percentile}.json"

# 步骤2: 生成embedding（使用PCA + 统一的percentile截断和normalize）
pca_embeddings, normalized_embeddings, pca_model, pca_color_model = generate_embeddings_with_pca_percentile_clipping(
    train_rep, 
    pca_dim=50, 
    percentile=args.percentile,
    pca_model_path=args.pca_model_path,
    color_model_path=args.color_model_path,
    save_prefix=save_prefix,
    batch_id=args.batch_id,
    stats_file_path=stats_file_path  # 新增统计文件路径
)

# 步骤3: 可视化并保存
output_image, coord_color_file = visualize_and_save_embeddings(
    total_minx, total_maxx, total_miny, total_maxy,
    patch_adata['x'], patch_adata['y'], patch_adata['gene'], patch_adata['cell'],
    normalized_embeddings, save_prefix, args.target_width, args.target_height, args.batch_id)

# 步骤4: 采样并创建adata
final_adata = sample_spots_and_create_adata(
    patch_adata, pca_embeddings, normalized_embeddings, 
    sample_ratio=args.sample_ratio, save_prefix=save_prefix, batch_id=args.batch_id)

# 步骤5: 整理结果给思婕
results_dir = organize_results_for_delivery(
    save_prefix, args.dataset_name, args.split_slice, 
    args.x_min, args.x_max, args.y_min, args.y_max, args.batch_id)

print(f"\n=== 处理完成 ===")
print(f"结果目录: {results_dir}")
print(f"最终spot数量: {len(patch_adata)}")
print(f"AnnData形状: {final_adata.shape}")
print(f"统计信息文件: {stats_file_path}")